In [11]:
import requests
import json
import getpass # To securely get the API token

def check_api_token(jira_url, email, api_token):
    """
    Checks if the API token is valid by fetching current user details.

    Args:
        jira_url (str): The base URL of your Jira instance.
        email (str): The email address associated with your Jira account.
        api_token (str): Your Jira API token.

    Returns:
        bool: True if the token is valid, False otherwise.
    """
    print("\nChecking API token validity...")
    # Using the /myself endpoint is a good way to check authentication
    api_url = f"{jira_url.rstrip('/')}/rest/api/3/myself"
    headers = {
        "Accept": "application/json"
    }
    try:
        response = requests.get(
            api_url,
            headers=headers,
            auth=(email, api_token)
        )
        if response.status_code == 200:
            user_data = response.json()
            print(f"API token is valid. Authenticated as: {user_data.get('displayName', email)}")
            return True
        else:
            print(f"Failed to validate API token. Status Code: {response.status_code}")
            print("Response:", response.text)
            return False
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during API token validation: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred during API token validation: {e}")
        return False

def list_accessible_projects(jira_url, email, api_token):
    """
    Lists projects accessible with the given API token.

    Args:
        jira_url (str): The base URL of your Jira instance.
        email (str): The email address associated with your Jira account.
        api_token (str): Your Jira API token.
    """
    print("\nFetching accessible projects...")
    # This endpoint fetches projects the user can see.
    api_url = f"{jira_url.rstrip('/')}/rest/api/3/project/search?maxResults=200" # Fetch up to 200 projects
    headers = {
        "Accept": "application/json"
    }
    try:
        response = requests.get(
            api_url,
            headers=headers,
            auth=(email, api_token)
        )
        if response.status_code == 200:
            projects_data = response.json()
            projects = projects_data.get('values', [])
            if projects:
                print("Accessible projects:")
                for project in projects:
                    print(f"  - Name: {project.get('name')}, Key: {project.get('key')}")
            else:
                print("No projects found or accessible with this token.")
        else:
            print(f"Failed to fetch projects. Status Code: {response.status_code}")
            print("Response:", response.text)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching projects: {e}")
    except Exception as e:
        print(f"An unexpected error occurred while fetching projects: {e}")


def add_jira_label(jira_url, email, api_token, ticket_key, label_to_add):
    """
    Adds a label to a Jira ticket.

    Args:
        jira_url (str): The base URL of your Jira instance.
        email (str): The email address associated with your Jira account.
        api_token (str): Your Jira API token.
        ticket_key (str): The key of the Jira ticket.
        label_to_add (str): The label string to add to the ticket.
    """
    api_url = f"{jira_url.rstrip('/')}/rest/api/3/issue/{ticket_key}"
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    payload = json.dumps({
        "update": {
            "labels": [
                {"add": label_to_add}
            ]
        }
    })

    print(f"\nAttempting to add label '{label_to_add}' to ticket '{ticket_key}'...")
    try:
        response = requests.put(
            api_url,
            data=payload,
            headers=headers,
            auth=(email, api_token)
        )
        if response.status_code == 204:
            print(f"Successfully added label '{label_to_add}' to ticket '{ticket_key}'.")
        elif response.status_code == 200: # Some instances might return 200 OK with content
            print(f"Successfully added label '{label_to_add}' to ticket '{ticket_key}'. (Status: 200 OK)")
        else:
            print(f"Failed to add label to ticket '{ticket_key}'. Status Code: {response.status_code}")
            print("Response:", response.text)
            try:
                error_messages = response.json().get('errorMessages', [])
                errors = response.json().get('errors', {})
                if error_messages:
                    print("Error Messages:", ", ".join(error_messages))
                if errors:
                    print("Detailed Errors:", errors)
            except json.JSONDecodeError:
                print("Could not parse error response as JSON.")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the request for ticket '{ticket_key}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred for ticket '{ticket_key}': {e}")

if __name__ == "__main__":
    print("Jira Ticket Label Adder with Diagnostics")
    print("--------------------------------------")

    # --- Pre-filled variables ---
    jira_url = "https://yourorg.atlassian.net/"
    email = "email@email.com"
    # --- End of Pre-filled variables ---

    # API Token will still be prompted
    api_token = getpass.getpass("Enter your Jira API Token: ").strip()

   
    ticket_keys_list = ["ORG-1234"]  # Initialize as empty or add your default tickets here


    if not ticket_keys_list:
        print("\n--- Add Label to Tickets ---")
        print("No pre-filled tickets. Enter Jira Ticket Keys, one per line. Press Enter on an empty line when done:")
        while True:
            ticket_key_input = input().strip()
            if not ticket_key_input: # Break if the line is empty
                break
            ticket_keys_list.append(ticket_key_input.upper())
    # --- End of Pre-filled ticket list ---


    # Perform diagnostic checks
    if not jira_url or not email or not api_token:
        print("Jira URL, email, and API token are required. Exiting.")
    else:
        print(f"Using Jira URL: {jira_url}")
        print(f"Using Email: {email}")
        token_is_valid = check_api_token(jira_url, email, api_token)

        if token_is_valid:
            list_accessible_projects(jira_url, email, api_token)

            if not ticket_keys_list:
                print("No ticket keys provided. Exiting labeling process.")
            else:
                label_to_add = "label" # As per your request

                print(f"\nDetails for labeling:")
                print(f"  Ticket Keys: {', '.join(ticket_keys_list)}")
                print(f"  Label to add: {label_to_add}")

                confirm = input(f"\nProceed with adding the label '{label_to_add}' to {len(ticket_keys_list)} ticket(s)? (yes/no): ").strip().lower()
                if confirm == 'yes':
                    for ticket_key in ticket_keys_list:
                        add_jira_label(jira_url, email, api_token, ticket_key, label_to_add)
                else:
                    print("Labeling operation cancelled by the user.")
        else:
            print("\nCannot proceed with labeling due to API token validation failure.")

    print("\nScript finished.")
